In [19]:
import pandas as pd
import geopandas as gpd

In [20]:
url1 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
US_confirmed = pd.read_csv(url1, dtype={'UID': str})

In [21]:
US_confirmed.columns

Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20',
       '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20',
       '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20',
       '2/13/20', '2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20',
       '2/19/20', '2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20',
       '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20',
       '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20',
       '3/9/20', '3/10/20', '3/11/20', '3/12/20', '3/13/20', '3/14/20',
       '3/15/20', '3/16/20', '3/17/20', '3/18/20', '3/19/20', '3/20/20',
       '3/21/20', '3/22/20', '3/23/20', '3/24/20', '3/25/20', '3/26/20',
       '3/27/20', '3/28/20', '3/29/20', '3/30/20', '3/31/20', '4/1/20',
       '4/2/20', '4/3/20'],
 

In [22]:
# extract data from PA, MI,and NY
PA_confirmed = US_confirmed[US_confirmed.UID.str.contains('84042')]
MI_confirmed = US_confirmed[US_confirmed.UID.str.contains('84026')]
NY_confirmed = US_confirmed[US_confirmed.UID.str.contains('84036')]

In [23]:
# remove additional columns
PA_confirmed = PA_confirmed.iloc[:,[5]+list(range(11, len(PA_confirmed.columns), 1))]
MI_confirmed = MI_confirmed.iloc[:,[5]+list(range(11, len(MI_confirmed.columns), 1))]
NY_confirmed = NY_confirmed.iloc[:,[5]+list(range(11, len(NY_confirmed.columns), 1))]

In [24]:
# convert from wide to long format
PA_confirmed_long = pd.melt(PA_confirmed, id_vars=['Admin2'])

In [25]:
# rename column names
PA_confirmed_long.rename(columns={'Admin2': 'name', 'variable': 'time', 'value': 'confirmed_cases'}, inplace=True)

In [28]:
# convert time column to time type
PA_confirmed_long['time'] = pd.to_datetime(PA_confirmed_long['time'])

In [39]:
PA_confirmed_long['time'] = PA_confirmed_long['time'].dt.strftime('%Y-%m-%d'+'T00:00:00')

In [41]:
PA_confirmed_long.head(4)

,name,time,confirmed_cases
0,Adams,2020-01-22T00:00:00,0
1,Allegheny,2020-01-22T00:00:00,0
2,Armstrong,2020-01-22T00:00:00,0
3,Beaver,2020-01-22T00:00:00,0


In [42]:
# import county boundary geojson file
PA_county = gpd.read_file('./county_boundary/PA_County.geojson')

In [43]:
# remove additional columns in geojson
PA_county = PA_county.iloc[:,[2,19]]

In [44]:
# merge number of cases file into geojson
PA_county = PA_county.merge(PA_confirmed_long, on='name')

In [12]:
#
#PA_county.to_file("PA_county_cases.geojson", driver='GeoJSON')

In [46]:
PA_test = PA_county[PA_county['time'].isin(['2020-04-01T00:00:00', '2020-04-02T00:00:00'])]
PA_test.to_file("PA_county_twodayscases.geojson", driver='GeoJSON')

In [47]:
PA_test2 = PA_county[PA_county['time'].isin(['2020-04-02T00:00:00'])]
PA_test2.to_file("PA_county_0402cases.geojson", driver='GeoJSON')

In [48]:
PA_test2.shape

(67, 4)